In [1]:
import ssl
ssl._create_default_https_context = ssl._create_stdlib_context

In [7]:
import yt_dlp
url = 'https://www.youtube.com/watch?v=WaZHX6RhGLs'
ydl_opts = {}
with yt_dlp.YoutubeDL(ydl_opts) as ydl:
    info = ydl.extract_info(url, download=False)

    print(f'title: {info.get('title')}')
    print(f'duration: {info.get('duration')}s')
    print(f'uploader: {info.get('uploader')}')
    print(f'channel_url: {info.get('uploader_url')}')
    print(f'thumbnail: {info.get('thumbnail')}')
    print(f'view count: {info.get('view_count')}')

[youtube] Extracting URL: https://www.youtube.com/watch?v=WaZHX6RhGLs
[youtube] WaZHX6RhGLs: Downloading webpage


[youtube] WaZHX6RhGLs: Downloading android sdkless player API JSON
[youtube] WaZHX6RhGLs: Downloading web safari player API JSON


[youtube] WaZHX6RhGLs: Downloading m3u8 information


title: 「IVORY TOWER feat. SennaRin MV 龍族ver.」｜アニメ「龍族 -The Blazing Dawn-」
duration: 180s
uploader: アニプレックス チャンネル
channel_url: https://www.youtube.com/@aniplex
thumbnail: https://i.ytimg.com/vi/WaZHX6RhGLs/maxresdefault.jpg
view count: 4274094


# first try of yt downloader

In [9]:
import customtkinter as ctk
from tkinter import filedialog, messagebox
import yt_dlp
import os

def download_video():
    url = url_entry.get()
    folder = folder_path.get()

    if not url:
        messagebox.showwarning('error', 'please enter youtube url!')
        return
    if not folder:
        messagebox.showwarning('error', 'please choose a folder to save!')
        return
    
    try:
        ydl_opts={
            'format': 'best',
            'outtmpl': f'{folder}%(title)s.%(ext)s', # set route and filename
        }

        status_label.configure(text='downloading, please wait...', text_color='yellow')
        root.update() # update the window

        with yt_dlp.YoutubeDL(ydl_opts) as ydl:
            ydl.download([url])

        status_label.configure(text='successfully downloaded!', text_color='green')
        messagebox.showinfo('finish', 'successfully download the video!')
    except Exception as e:
        status_label.configure(text='download failed', text_color='red')
        messagebox.showerror('error', f'error occured: {str(e)}')
    
def select_folder():
    path = filedialog.askdirectory()
    if path:
        folder_path.set(path)

# set GUI
ctk.set_appearance_mode('dark')
root = ctk.CTk()
root.title('YouTube Video Downloader')
root.geometry('500x300')

# url input
url_label = ctk.CTkLabel(root, text='YouTube url:')
url_label.pack(pady=(20, 0))
url_entry = ctk.CTkEntry(root, width=400, placeholder_text='input the url')
url_entry.pack(pady=10)

# folder choose
folder_path = ctk.StringVar(value=os.getcwd()) # set the current folder as default
folder_frame = ctk.CTkFrame(root, fg_color='transparent')
folder_frame.pack(pady=10)

folder_label = ctk.CTkEntry(folder_frame, textvariable=folder_path, width=300)
folder_label.pack(side='left', padx=5)

folder_btn = ctk.CTkButton(folder_frame, text='choose folder', command=select_folder, width=80)
folder_btn.pack(side='left')

# download button
download_btn = ctk.CTkButton(root, text='start download', command=download_video, fg_color='green', hover_color='darkgreen')
download_btn.pack(pady=20)

# status showing
status_label = ctk.CTkLabel(root, text='')
status_label.pack()

root.mainloop()


[youtube] Extracting URL: https://www.youtube.com/watch?v=WaZHX6RhGLs
[youtube] WaZHX6RhGLs: Downloading webpage


[youtube] WaZHX6RhGLs: Downloading android sdkless player API JSON
[youtube] WaZHX6RhGLs: Downloading web safari player API JSON


[youtube] WaZHX6RhGLs: Downloading m3u8 information


[info] WaZHX6RhGLs: Downloading 1 format(s): 96
[download] Sleeping 7.00 seconds as required by the site...
[hlsnative] Downloading m3u8 manifest
[hlsnative] Total fragments: 37
[download] Destination: d:\personal\side_project\yt_download「IVORY TOWER feat. SennaRin MV 龍族ver.」｜アニメ「龍族 -The Blazing Dawn-」.mp4
[download] 100% of   52.10MiB in 00:00:22 at 2.31MiB/s                  


# second try of yt downloader

In [1]:
import customtkinter as ctk
from tkinter import filedialog, messagebox
import yt_dlp
import threading
import os
import sys

class YTDownloaderApp(ctk.CTk):
    def __init__(self):
        super().__init__()

        self.title('YouTube Video/Music Downloader')
        self.geometry('600x550')
        ctk.set_appearance_mode('dark')

        # --UI setting--
        # title
        self.header = ctk.CTkLabel(self, text='YouTube Downloader', font=('Microsoft JhengHei', 24, 'bold'))
        self.header.pack(pady=20)

        # url enter area (multiple lines)
        self.url_label = ctk.CTkLabel(self, text='Please enter URL (one per line):')
        self.url_label.pack(anchor='w', padx=50)
        self.url_textbox = ctk.CTkTextbox(self, width=500, height=120)
        self.url_textbox.pack(pady=5)

        # format and quality selection area
        self.option_frame = ctk.CTkFrame(self, fg_color='transparent')
        self.option_frame.pack(pady=10)
        
        self.type_var = ctk.StringVar(value='mp4')
        self.type_menu = ctk.CTkOptionMenu(self.option_frame, values=['mp4', 'mp3'], variable=self.type_var, width=120)
        self.type_menu.pack(side='left', padx=10)

        self.quality_var = ctk.StringVar(value='best quality')
        self.quality_menu = ctk.CTkOptionMenu(self.option_frame, values=['best quality', '1080p', '720p', '480p'], variable=self.quality_var, width=120)
        self.quality_menu.pack(side='left', padx=10)

        # check box for loundnorm
        self.check_frame = ctk.CTkFrame(self, fg_color='transparent')
        self.check_frame.pack(pady=5)

        self.norm_var = ctk.BooleanVar(value=False)
        self.norm_checkbox = ctk.CTkCheckBox(
            self.check_frame,
            text='Automatic volume unification (will increase processing time)',
            variable=self.norm_var,
            font=('Microsoft JhengHei', 12)
        )
        self.norm_checkbox.pack()

        # save path
        self.path_frame = ctk.CTkFrame(self, fg_color='transparent')
        self.path_frame.pack(pady=10)
        self.path_var = ctk.StringVar(value=os.getcwd())
        self.path_entry = ctk.CTkEntry(self.path_frame, textvariable=self.path_var, width=350)
        self.path_entry.pack(side='left', padx=5)
        self.path_btn = ctk.CTkButton(self.path_frame, text='change folder', command=self.select_path, width=100)
        self.path_btn.pack(side='left')

        # download button
        self.download_btn = ctk.CTkButton(self, text='start downloading', command=self.start_download_thread,
                                          fg_color='#24a0ed', hover_color='#007aac', font=('Microsoft JhengHei', 16, 'bold'))
        self.download_btn.pack(pady=20)

        # showing status
        self.status_label = ctk.CTkLabel(self, text='wait for command...', text_color='gray')
        self.status_label.pack()

    def select_path(self):
        path = filedialog.askdirectory()
        if path:
            self.path_var.set(path)


    def start_download_thread(self):
        # get all the urls and filter out blank lines
        urls = self.url_textbox.get('1.0', 'end-1c').split('\n')
        urls = [url.strip() for url in urls if url.strip()]

        folder = self.path_var.get()
        file_type = self.type_var.get()
        quality = self.quality_var.get()

        do_norm = self.norm_var.get()

        if not urls:
            messagebox.showwarning('warning', 'atleat enter one url')
            return
        
        # diable the button to prevent repeated clicks
        self.download_btn.configure(state='diabled')
        # start thread to download
        thread = threading.Thread(
            target=self.run_downloads, 
            args=(urls, folder, file_type, quality, do_norm))
        thread.daemon = True # 設定為守護執行緒，視窗關閉時會自動結束
        thread.start()

    def get_tool_path(self, filename):
        # 如果是打包後的環境，PyInstaller 會將檔案解壓縮到 _MEIPASS 臨時目錄
        if hasattr(sys, '_MEIPASS'):
            return os.path.join(sys._MEIPASS, filename)
        # 如果是開發環境，則尋找當前目錄下的 ffmpeg.exe
        return os.path.abspath(filename)
    
    def run_downloads(self, urls, folder, file_type, quality, do_norm):
        # set the args of ty-dlp
        for i, url in enumerate(urls):
            self.status_label.configure(text=f'now downloading {i+1}/{len(urls)} videos...', text_color='yellow')
            
            # set the download args (core)
            ydl_opts = self.get_ydl_options(folder, file_type, quality, do_norm, url)

            try:
                with yt_dlp.YoutubeDL(ydl_opts) as ydl:
                    ydl.download([url])
            except Exception as e:
                print(f'Error: {e}')
                continue

        self.after(0, self.finish_ui_update, len(urls))

    def finish_ui_update(self, count):
        # 這個函數會在主執行緒執行，所以可以安全呼叫 GUI 元件
        self.status_label.configure(text='All Downloading Tasks Finished!', text_color='green')
        self.download_btn.configure(state='normal')
        messagebox.showinfo('Finished', f'Complete {count} downloading tasks')

    def my_hook(self, d):
        if d['status'] == 'downloading':
            index = d.get('info_dict', {}).get('playlist_index')
            total = d.get('info_dict', {}).get('n_entries')
            if index and total:
                self.status_label.configure(text=f'Now Downloading the Playlist: {index}/{total}')

    def get_ydl_options(self, folder, file_type, quality, do_norm, url):
        # basic saving path and file format
        ffmpeg_path = self.get_tool_path('ffmpeg.exe')
        qjs_path = self.get_tool_path('qjs.exe')

        if 'list=' in url:
            template = f'{folder}/%(playlist_title)s/%(playlist_index)s -%(title)s.%(ext)s'
        else:
            template = f'{folder}/%(title)s.%(ext)s'

        opts = {
            'outtmpl': template,
            'quiet': True,
            'no_warnings': True,
            'ffmpeg_location': ffmpeg_path,
            'extra_js_runtimes': [f'quickjs:{qjs_path}'],
            'prefer_ffmpeg': True,
            'noplaylist': False,
            'ignoreerrors': True,
            'progress_hooks': [self.my_hook],
        }

        # volume normalize
        if do_norm:
            opts['postprocessor_args'] = {
                'ffmpeg': ['-af', 'loudnorm=I=-16:TP=-1.5:LRA=11']
            }

        if file_type == 'mp3':
            opts.update({
                'format': 'bestaudio/best',
                'postprocessors': [{
                    'key': 'FFmpegExtractAudio',
                    'preferredcodec': 'mp3',
                    'preferredquality': '192',
                }],

            })
        else:
            q_map = {
                '1080p': 'bestvideo[height<=1080][ext=mp4]+bestaudio[ext=m4a]/best[height<=1080]',
                '720p': 'bestvideo[height<=720][ext=mp4]+bestaudio[ext=m4a]/best[height<=720]',
                '480p': 'bestvideo[height<=480][ext=mp4]+bestaudio[ext=m4a]/best[height<=480]',
                'best quality': 'bestvideo+bestaudio/best',
            }
            opts['format'] = q_map.get(quality, 'bestvideo+bestaudio/best')
            opts['merge_utput_format'] = 'mp4'

        return opts
    
if __name__ == '__main__':
    app = YTDownloaderApp()
    app.mainloop()

